In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
calendar = pd.read_csv('calendar.csv')
listings = pd.read_csv('listings.csv')
reviews = pd.read_csv('reviews.csv')

# EDA

#### Explore the data
3 tables:
- Calendar: listing_id, data, available, price
- Listings: id, name, summary, description, city, neighbourhood, etc.
- Reviews: listing_id, data, reviewer_name, comments

In [11]:
calendar.head()

,listing_id,date,available,price
0,12147973,2017-09-05,f,NaN
1,12147973,2017-09-04,f,NaN
2,12147973,2017-09-03,f,NaN
3,12147973,2017-09-02,f,NaN
4,12147973,2017-09-01,f,NaN


In [12]:
listings.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",...,NaN,f,NaN,NaN,f,moderate,f,f,1,NaN
1,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,"The room is in Roslindale, a diverse and prima...",...,9.0,f,NaN,NaN,t,moderate,f,f,1,1.30
2,6976,https://www.airbnb.com/rooms/6976,20160906204935,2016-09-07,Mexican Folk Art Haven in Boston,"Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...","Come stay with a friendly, middle-aged guy in ...",none,The LOCATION: Roslindale is a safe and diverse...,...,10.0,f,NaN,NaN,f,moderate,t,f,1,0.47
3,1436513,https://www.airbnb.com/rooms/1436513,20160906204935,2016-09-07,Spacious Sunny Bedroom Suite in Historic Home,Come experience the comforts of home away from...,Most places you find in Boston are small howev...,Come experience the comforts of home away from...,none,Roslindale is a lovely little neighborhood loc...,...,10.0,f,NaN,NaN,f,moderate,f,f,1,1.00
4,7651065,https://www.airbnb.com/rooms/7651065,20160906204935,2016-09-07,Come Home to Boston,"My comfy, clean and relaxing home is one block...","Clean, attractive, private room, one block fro...","My comfy, clean and relaxing home is one block...",none,"I love the proximity to downtown, the neighbor...",...,10.0,f,NaN,NaN,f,flexible,f,f,1,2.25


In [13]:
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,1178162,4724140,2013-05-21,4298113,Olivier,My stay at islam's place was really cool! Good...
1,1178162,4869189,2013-05-29,6452964,Charlotte,Great location for both airport and city - gre...
2,1178162,5003196,2013-06-06,6449554,Sebastian,We really enjoyed our stay at Islams house. Fr...
3,1178162,5150351,2013-06-15,2215611,Marine,The room was nice and clean and so were the co...
4,1178162,5171140,2013-06-16,6848427,Andrew,Great location. Just 5 mins walk from the Airp...


##### Calendar
- Each listing_id correspond to the id in Listings.
- Calendar includes the room occupied/vacant infomation of each date.

In [6]:
# total observations of Calendar
len(calendar)
# total oberservations with price
sum(calendar.price.isnull())

1308890

In [32]:
calendar['date'] = pd.to_datetime(calendar['date']).dt.strftime('%Y:%M:%D')
sns.scatterplot(calendar['date'], calendar['available'])
plt.show()

ValueError: hour must be in 0..23

# 前處理

# 過程
- 目標是預測價錢，所以先把在 cal 內有 price 的資料撈出來
- 做一些簡單的前處理後直接用 linear model 來做預測
- 之後迭代模型，到一定的效能之後看何者跟預測結果的相關性比較高

# 想法
- 能夠把有意義的文字資料全部串起來然後送進 Transformer 中看看預測的效能嗎？

In [16]:
calendar.head()

,listing_id,date,available,price
0,12147973,2017-09-05,f,NaN
1,12147973,2017-09-04,f,NaN
2,12147973,2017-09-03,f,NaN
3,12147973,2017-09-02,f,NaN
4,12147973,2017-09-01,f,NaN


In [37]:
reviews.columns

Index(['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments'], dtype='object')

In [41]:
calendar.head(500)

,listing_id,date,available,price
0,12147973,2017:00:09/05/17,f,NaN
1,12147973,2017:00:09/04/17,f,NaN
2,12147973,2017:00:09/03/17,f,NaN
3,12147973,2017:00:09/02/17,f,NaN
4,12147973,2017:00:09/01/17,f,NaN
...,...,...,...,...
495,3075044,2017:00:04/14/17,t,$75.00
496,3075044,2017:00:04/13/17,t,$65.00
497,3075044,2017:00:04/12/17,t,$65.00
498,3075044,2017:00:04/11/17,t,$65.00


In [24]:
sum(calendar.price.isnull())

665853

In [34]:
len(calendar[calendar['available'] == 'f'])

665853

In [43]:
t = '2017:00:04/14/17'
re.sub(r'\d\d\d\d:\d\d:', '', t)

'04/14/17'

In [44]:
format_time = lambda re.sub(r'\d\d\d\d:\d\d:', '', calendar['data']) : calendar['data']

SyntaxError: invalid syntax (<ipython-input-44-ebf7852e4eef>, line 1)